# Scikit-Learn course alapján
#### https://youtu.be/pqNCD_5r0IU?t=9151

In [ ]:
import mnist
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import binarize
from skimage.filters import threshold_local
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

In [ ]:
xtrain = mnist.train_images()
ytrain = mnist.train_labels()

xtest = mnist.test_images()
ytest = mnist.test_labels()

In [ ]:
xtrain = xtrain.reshape((-1, 28*28))
xtest = xtest.reshape((-1,28*28))

In [ ]:
unique_classes, counts = np.unique(ytrain, return_counts=True)

# Kiíratás az egyes osztályokhoz tartozó minták számáról
for i, label in enumerate(unique_classes):
    print(f"Osztály {label}: {counts[i]} darab minta")

samples_to_display = []
for class_label in unique_classes:
    indices = np.where(ytrain == class_label)[0]
    sample_indices = np.random.choice(indices, size=1, replace=False)  # Válasszunk 5 mintát véletlenszerűen
    samples_to_display.extend(sample_indices)

fig, axes = plt.subplots(1, len(samples_to_display), figsize=(15, 2))
for i, idx in enumerate(samples_to_display):
    axes[i].imshow(xtrain[idx].reshape(28, 28), cmap='gray')
    axes[i].set_title(f"Class: {ytrain[idx]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

A meglenített minták háttere fekete, emiatt a későbbíekben látható lesz, hogy a színek invertálásával pontosabb eredményt kapunk.

In [ ]:
xtrain = np.array(xtrain/256)
xtest = np.array(xtest/256)

In [ ]:
# Validációs adathalmaz létrehozása
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.5, random_state=42, stratify=ytrain)

# MLP osztály inicializálása
clf = MLPClassifier(solver='adam', activation='relu', hidden_layer_sizes=(64, 64), max_iter=1)

# Pontosságok tárolása
train_accuracy_list = []
val_accuracy_list = []

# Lépésenkénti tanítás és pontosságok nyomon követése
for i in range(100):
    #print(f'{i + 1} / 100')
    clf.partial_fit(xtrain, ytrain, classes=np.unique(ytrain))
    
    # Pontosság mérése a tanító adathalmazon
    ytrain_pred = clf.predict(xtrain)
    train_accuracy = accuracy_score(ytrain, ytrain_pred)
    train_accuracy_list.append(train_accuracy)
    
    # Pontosság mérése a validációs adathalmazon
    yval_pred = clf.predict(xval)
    val_accuracy = accuracy_score(yval, yval_pred)
    val_accuracy_list.append(val_accuracy)

# Pontosságok megjelenítése
plt.plot(train_accuracy_list, label='Tanító adathalmaz')
plt.plot(val_accuracy_list, label='Validációs adathalmaz')
plt.xlabel('Iterációk')
plt.ylabel('Pontosság')
plt.legend()
plt.show()


In [ ]:
prediction = clf.predict(xtest)

In [ ]:
acc = confusion_matrix(ytest, prediction)

In [ ]:
def accuracy(cm):
    diag = cm.trace()
    elements = cm.sum()
    return diag/elements

print(accuracy(acc))

In [ ]:
def image_to_bin(img):
    data = list(img.getdata())
    for i in range(len(data)):
        data[i] = 255 - data[i]
    data = np.array(data)/256.0
    return data

# Gimpben rajzolt számjegyek

### Itt sokkal pontosabbak az eredmények, valószínűleg ezek jobban hasonlítanak a betanított számjegyekhez.

In [ ]:
image_folder = "drawn_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Kézzel írott számjegyek

In [ ]:
image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = Image.open(file_path)
        binimg = image_to_bin(image)
        p = clf.predict([binimg])
        images.append(image)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Lokális küszöböléssel
### kézi implementáció

In [ ]:
import cv2

def local_threshold_mean(image, kernel_size):
    # Átlagszűrő alkalmazása a küszöbérték meghatározásához
    blurred_image = cv2.blur(image, (kernel_size, kernel_size))
    
    # Konvolúció az eredeti képpel
    convolved_image = cv2.filter2D(image, -1, np.ones((kernel_size, kernel_size), dtype=np.float32) / (kernel_size * kernel_size))
    
    # Lokális küszöbölés az átlag alapján
    thresholded_image = np.where(image > convolved_image, 255, 0).astype(np.uint8)
    
    return thresholded_image

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása
        binimg = local_threshold_mean(image, kernel_size=30)
        p = clf.predict([binimg.flatten()])
        images.append(binimg)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


## Kézi implementáció színek invertálásával

In [ ]:
def local_threshold_mean(image, kernel_size):
    # Átlagszűrő alkalmazása a küszöbérték meghatározásához
    blurred_image = cv2.blur(image, (kernel_size, kernel_size))
    
    # Konvolúció az eredeti képpel
    convolved_image = cv2.filter2D(image, -1, np.ones((kernel_size, kernel_size), dtype=np.float32) / (kernel_size * kernel_size))
    
    # Lokális küszöbölés az átlag alapján
    thresholded_image = np.where(image > convolved_image, 255, 0).astype(np.uint8)
    
    return thresholded_image

def invert_colors(img):
    return cv2.bitwise_not(img)

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása
        binimg = local_threshold_mean(image, kernel_size=100)
        inverted_img = invert_colors(binimg)
        p = clf.predict([inverted_img.flatten()])
        images.append(inverted_img)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()

# Lokális küszöbölés 
### scikit-image

In [ ]:
from skimage.filters import threshold_local

def local_thresholding(img):
    # Lokális küszöbölés alkalmazása
    block_size = 35  # A blokk mérete, amelyben a küszöbölés alkalmazódik
    local_threshold = threshold_local(img, block_size, offset=5, method='gaussian')
    binary_image = (img > local_threshold).astype(np.uint8) * 255

    return binary_image

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása
        binimg = local_thresholding(image)
        p = clf.predict([binimg.flatten()])
        images.append(binimg)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()


## színek invertálásával

In [ ]:
def local_thresholding(img):
    # Lokális küszöbölés alkalmazása
    block_size = 35  # A blokk mérete, amelyben a küszöbölés alkalmazódik
    local_threshold = threshold_local(img, block_size, offset=30, method='gaussian')
    binary_image = (img > local_threshold).astype(np.uint8) * 255

    # Kép inverzének előállítása
    inverted_image = cv2.bitwise_not(binary_image)

    return inverted_image

image_folder = "handwritten_digits"
images = []
predictions = []

# Képek beolvasása és osztályozás végrehajtása
for filename in os.listdir(image_folder):
    if filename.endswith(".png"):
        file_path = os.path.join(image_folder, filename)
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Képek szürkeárnyalatosan beolvasása
        binimg = local_thresholding(image)
        p = clf.predict([binimg.flatten()])
        images.append(binimg)
        predictions.append(p)

fig, axes = plt.subplots(2, 5, figsize=(10, 4))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='gray')
    ax.set_title("Prediction: {}".format(predictions[i]))

plt.tight_layout()
plt.show()
